In [1]:
from pathlib import Path
import tensorflow.keras as keras
import tensorflow as tf
import sys
import json
import os
import music21 as m21
import numpy as np
from typing import List
import matplotlib.pyplot as plt

In [2]:
class FileHelper:
    def __init__(self):
        pass

    def load_file_as_str(self, file_path: str) -> str:
        """
        Loads the file at a specificed path, returns an error otherwise
        
        :param file_path (str): The path to the file you want to load
        :return _file a str of the contents of the file
        """
        _file = ""
        try:
            with open(file_path, "r") as fp:
                _file = fp.read()
            return _file
        except Exception:
            raise Exception(f"Error reading file at: {file_path}")

    # TODO: Might not need this
    def readBytes(self, file_path) -> bytes:
        try:
            with open(file_path, "rb") as fb:
                file_bytes = fb.read()
                return file_bytes
        except Exception:
            raise Exception(f"Error trying to read bytes at: {file_path}")
            
    def loadJSON(self, file_path) -> dict:
        try:
            with open(file_path) as json_data:
                data = json.load(json_data)
            return data
        except Exception:
            raise Exception(f"Could not open file located at: {file_path}")

In [3]:
# durations are expressed in quarter length
ACCEPTABLE_DURATIONS = [
    0.25, # 16th note
    0.5, # 8th note
    0.75,
    1.0, # quarter note
    1.5,
    2, # half note
    3,
    4 # whole note
]

class MusicHelper:
    def __init__(self, file_helper: FileHelper, acceptable_durations=ACCEPTABLE_DURATIONS):
        """
        :param file_helper (FileHelper): This is a file_helper object to help with writing/reading files
        :param acceptable_durations (List[float]): This is the range of notes that are ok to add for the model
        :return MusicHelper: This is the constructor for the class
        """
        print("MusicHelper has been created...")
        self.acceptable_durations = acceptable_durations
        self._file_helper = file_helper

    def load_songs(self, dataset_path: str, file_type: str):
        """
        Loads all pieces of a specific file type

        :param dataset_path (str): Path to dataset
        :param file_stype (str): The file type you want to load
        :return songs (list of m21 streams): List containing all pieces
        """
        print("Loading songs....")
        songs = []
        len_file_ext = len(file_type)

        # go through all the files in dataset and load them with music21
        for path, _, files in os.walk(dataset_path):
            for file in files:

                # consider only files of the target type
                if file[-len_file_ext:] == file_type:
                    try:
                        song = m21.converter.parse(os.path.join(path, file))
                        if self.monophonic(song):
                            songs.append(song)
                    except Exception:
                        print(f"Error processing file located at: {os.path.join(path, file)}")
        return songs
    
    def monophonic(self, stream) -> bool:
        try:
            length = len(m21.instrument.partitionByInstrument(stream).parts)
        except:
            length = 0
        return length == 1

    def transpose_song(self, song, major_key: str, minor_key: str):
        """
        Transposes song to a specified major/minor key.
        If the song is in a major scale it is transposed to the specified key, vice versa

        Defaults are in place in case the programmer forgets them

        :param song (m21 stream): The song to transpose
        :param major_key (str): The musical major key you want to transpose to
        :param minor_key (str): The musical minor key you want to transpose to
        :return transposed_song (m21 stream):
        """
        #print("Transposing song....")
        # get key from the song
        parts = song.getElementsByClass(m21.stream.Part)
        measures_part0 = parts[0].getElementsByClass(m21.stream.Measure)
        key = song.analyze("key")

        # get interval for transposition. E.g., Bmaj -> Cmaj
        if key.mode == "major":
            interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
        elif key.mode == "minor":
            interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))

        # transpose song by calculated interval
        tranposed_song = song.transpose(interval)
        return tranposed_song.chordify()

    def encode_song(self, song, time_step=0.25):
        """
        Converts a score into a time-series-like music representation. Each item in the encoded list represents 'min_duration'
        quarter lengths. The symbols used at each step are: integers for MIDI notes, 'r' for representing a rest, and '_'
        for representing notes/rests that are carried over into a new time step. Here's a sample encoding:

            ["r", "_", "60", "_", "_", "_", "72" "_"]

        :param song (m21 stream): Piece to encode
        :param time_step (float): Duration of each time step in quarter length
        :return:
        """
        #print("Encoding Song...")
        encoded_song = []

        for event in song.flat.notesAndRests:

            # handle notes
            if isinstance(event, m21.note.Note):
                symbol = event.pitch.midi # 60
            # handle rests
            elif isinstance(event, m21.note.Rest):
                symbol = "r"
            elif isinstance(event, m21.chord.Chord):
                symbol = '.'.join(str(n) for n in event.normalOrder)

            # convert the note/rest into time series notation
            steps = int(event.duration.quarterLength / time_step)
            for step in range(steps):

                # if it's the first time we see a note/rest, let's encode it. Otherwise, it means we're carrying the same
                # symbol in a new time step
                if step == 0:
                    encoded_song.append(symbol)
                else:
                    encoded_song.append("_")

        # cast encoded song to str
        encoded_song = " ".join(map(str, encoded_song))

        return encoded_song

    def convert_songs_to_int(self, songs: str, mapping_path:str):
        """ 
        Maps the symbol in the song to an integer as dictated by the mappings.json file you create
        earlier in training

        :param songs (str): The giant song string you are currently using
        :mapping_path (str): Path to your mappins.json file
        :return int_songs
        """
        int_songs = []
        
        # load mappings
        # TODO: Make the file_helper do this
        with open(mapping_path, "r") as fp:
            mappings = json.load(fp)

        # cast songs string to a list
        songs = songs.split()

        # map sings to int
        for symbol in songs:
            # TODO: Add some defensive coding strats here if key !exists 
            int_songs.append(mappings[symbol])

        return int_songs

    def create_mapping(self, songs: str, mapping_path: str) -> None:
        """
        Creates a json file that maps the symbols in the song dataset onto integers
        This mappins file is EXTREMELY important for further training, dont lose it!

        :param songs (str): String with all songs
        :param mapping_path (str): Path where to save mapping
        :return:
        """
        mappings = {}

        # identify the vocabulary
        songs = songs.split()
        vocabulary = list(set(songs))

        # create mappings
        for i, symbol in enumerate(vocabulary):
            mappings[symbol] = i

        # save voabulary to a json file
        # TODO: Have the file_helper do this
        with open(mapping_path, "w+") as fp:
            json.dump(mappings, fp, indent=4)

    def generate_training_sequences(self, sequence_length: int, single_file_dataset_path: str, mapping_path: str):
        """ 
        This creates representations of our data that can now be fed to an LSTM model

        :param sequence_length (int): The length of the "sliding window" we're using to refeed samples
        :param single_file_dataset_path (str): The file generated by create_single_file_dataset
        :param mapping_path (str): The mapping file that maps symbols to ints
        :return inputs, targets (3D Numpy Array): This is a 3d Numpy array/tensor for the model
        """
        # What this is trying to create in the model:
        # [11, 12, 13, 14, ...] -> inputs:[11, 12],  target:[13]

        # load songs and map them to int
        songs = self._file_helper.load_file_as_str(single_file_dataset_path)
        int_songs = self.convert_songs_to_int(songs, mapping_path)

        # generate the training sequences
        # 100 symbols, seq_len = 64, 100 - 64 = 36 sequences we can generate
        inputs = []
        targets = []
        num_sequences = len(int_songs) - sequence_length

        # This loops generates the training slices plus the target to predict
        for i in range(num_sequences):
            inputs.append(int_songs[i:i+sequence_length])
            targets.append(int_songs[i+sequence_length])

        # one-hot encode the sequences
        # inputs: (# of sequences, sequence length, vocabulary size)
        vocabulary_size = len(set(int_songs))
        inputs = keras.utils.to_categorical(inputs, num_classes=vocabulary_size, dtype=np.uint8)
        targets = np.array(targets)

        # Inputs will be a 3D Numpy array as demonstrated above
        return inputs, targets


    def create_single_file_dataset(self, encoded_song_path: str, file_dataset_path: str, sequence_length: int) -> str:
        """
        Generates a file collating all the encoded songs and adding new piece delimiters. It then saves that to disc and returns the song

        :param dataset_path (str): Path to folder containing the encoded songs
        :param file_dataset_path (str): Path to file for saving songs in single txt file
        :param sequence_length (int): # of time steps to be considered for training
        :return songs (str): String containing all songs in dataset + delimiters
        """

        new_song_delimiter = "/ " * sequence_length
        songs = ""

        # load encoded songs and add delimiters
        for path, _, files in os.walk(encoded_song_path):
            for file in files:
                file_path = os.path.join(path, file)
                song = self._file_helper.load_file_as_str(file_path)
                songs = songs + song + " " + new_song_delimiter

        # remove empty space from last character of string
        songs = songs[:-1]

        # save string that contains all the dataset
        # TODO: Move to file helper + add defensive checks
        with open(file_dataset_path, "w+") as fp:
            fp.write(songs)

        return songs

    def preprocess_songs(self, dataset_path: str, song_txt_path: str, major_key: str, minor_key: str, file_type: str) -> None:
        """
        A method that encompasses many of the preprocessing operations needed for converting
        the songs to a helpful representation for our RNN/LSTM/Time Series centric models

        :param dataset_path (str): The complete path to the dataset you want to load
        :param song_text_path (str): The complete path where the song_txt file will be saved
        :param major_key (str): What major key we transpose songs to
        :param minor_key (str): What minor key we transpose songs to
        :return None: This method returns nothing
        """

        # load folk songs
        #print("Loading songs...")
        songs = self.load_songs(dataset_path, file_type)
        print(f"Loaded {len(songs)} songs.")

        for i, song in enumerate(songs):
            # transpose songs to the major/minor key we want
            song = self.transpose_song(song, major_key, minor_key)

            # encode songs with music time series representation
            encoded_song = self.encode_song(song)

            # save songs to text file
            save_path = os.path.join(song_txt_path, str(i))
            # TODO: Have file helper do this + defensive checks
            with open(save_path, "w+") as fp:
                fp.write(encoded_song)

    # TODO: Ensure this is as genric as possible and applicate 
    def song_data_pipeline(self, pipeline_config: dict) -> None:
        """ 
        A single method that encapsulates the entire preprocessing pipeline for files. 

        :param pipeline_config (dict): A dict containing all the parameters and arguments for the methods being called.
        :return None: This method returns nothing
        """

        # TODO: Add better defensive coding, throughout the entire method really
        if (len(pipeline_config.keys()) < 6):
             raise Exception("Not enough keys, returning...")
        
        print("Entering song preprocessing...")
        self.preprocess_songs(
           pipeline_config['DATASET_PATH'], 
           pipeline_config['ENCODED_SONG_PATH'],
           pipeline_config['MAJOR_KEY'],
           pipeline_config['MINOR_KEY'],
           pipeline_config['FILE_TYPE']
        )
        
        songs = self.create_single_file_dataset(
            pipeline_config['ENCODED_SONG_PATH'],
           pipeline_config['SINGLE_FILE_DATASET_PATH'],
           pipeline_config['SEQUENCE_LENGTH']
        )

        self.create_mapping(songs, pipeline_config['MAPPING_PATH'])
        print("Finished preprocessing songs!")

        print("Generating training data...")
        inputs, targets = self.generate_training_sequences(
            pipeline_config['SEQUENCE_LENGTH'],
            pipeline_config['SINGLE_FILE_DATASET_PATH'],
            pipeline_config['MAPPING_PATH']
        )

        print("Finishing creating training data. Now returning....")
        return inputs, targets

In [4]:
# TODO: Make the output neurons return from the music_helper where the data processing
# actually happens
LOSS_FUNC = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.001
BATCH_SIZE = 16
NUM_NEURONS = [256] # number of neurons in the eternal layers
ROOT_PATH = Path.cwd()
SAVE_MODEL_PATH = f"{ROOT_PATH}/JazzDrums.h5"

# Used with the music helper class
pipeline_config = {
    'DATASET_PATH': f"{ROOT_PATH}/data",
    'ENCODED_SONG_PATH': f"{ROOT_PATH}/processed_songs/",
    'MAJOR_KEY': "C",
    'MINOR_KEY': "A",
    'SINGLE_FILE_DATASET_PATH': f"{ROOT_PATH}/massive_song_file_data.txt",
    'MAPPING_PATH': f"{ROOT_PATH}/song_mappings.json",
    'SEQUENCE_LENGTH': 64,
    'FILE_TYPE': "mid",
    'STAGE': 0
}

In [5]:
file_helper = FileHelper()
music_helper = MusicHelper(file_helper)

MusicHelper has been created...


In [6]:
inputs, targets = music_helper.song_data_pipeline(
            pipeline_config
        )

Entering song preprocessing...
Loading songs....
Loaded 400 songs.
Finished preprocessing songs!
Generating training data...
Finishing creating training data. Now returning....


In [7]:
def build_model(output_neurons, num_neurons, loss, learning_rate):

    # Create the model architecture, functionally!
    # If we say None for the shape, it lets us have any length input for the time steps
    # This is important because we might wanna feed it different length premade sequences
    input = keras.layers.Input(shape=(None, output_neurons))
    x = keras.layers.LSTM(num_neurons[0]*2)(input) # This links the layers together
    x = keras.layers.Dropout(0.2)(x)
    dense = keras.layers.Dense(num_neurons[0], activation="relu")(x)
    dense = keras.layers.Dropout(0.3)(dense)
    dense = keras.layers.Dense(num_neurons[0]//2, activation="relu")(dense)
    dense = keras.layers.Dropout(0.3)(dense)
    output = keras.layers.Dense(output_neurons, activation="softmax")(dense)

    # compile the model
    model = keras.Model(input, output)
    model.compile(loss=loss,
                optimizer=keras.optimizers.Adam(lr=learning_rate),
                metrics=["accuracy"])

    model.summary()

    return model

In [9]:
# set any callbacks
checkpoint_path = f"{ROOT_PATH}/checkpoints/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [10]:
OUTPUT_NEURONS = len(file_helper.loadJSON("./song_mappings.json").keys())
OUTPUT_NEURONS # Equal to vocabulary size from the mapping.json

262

NameError: name 'model' is not defined

In [12]:
model = build_model(OUTPUT_NEURONS, NUM_NEURONS, LOSS_FUNC, LEARNING_RATE)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 262)]       0         
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1587200   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [13]:
model.load_weights(checkpoint_path)

In [14]:
TOTAL_EPOCHS = 180
model.fit(inputs, targets, epochs=TOTAL_EPOCHS, batch_size=BATCH_SIZE, callbacks=[cp_callback])

# save the model
model.save(SAVE_MODEL_PATH)

Epoch 1/180
2625/2625 [==============================] - ETA: 0s - loss: 1.9000 - accuracy: 0.6784
Epoch 00001: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 46s 17ms/step - loss: 1.9000 - accuracy: 0.6784
Epoch 2/180
2623/2625 [============================>.] - ETA: 0s - loss: 1.3683 - accuracy: 0.6941
Epoch 00002: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 1.3683 - accuracy: 0.6942
Epoch 3/180
2623/2625 [============================>.] - ETA: 0s - loss: 1.2785 - accuracy: 0.7234
Epoch 00003: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 1.2783 - accuracy: 0.7235
Epoch 4/180
2625/2625 [==============================] - ETA: 0s - loss: 1.2160 - accuracy: 0.7422
Epoch 0

2625/2625 [==============================] - 46s 18ms/step - loss: 0.6606 - accuracy: 0.8381
Epoch 29/180
2623/2625 [============================>.] - ETA: 0s - loss: 0.6559 - accuracy: 0.8377
Epoch 00029: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 46s 17ms/step - loss: 0.6557 - accuracy: 0.8378
Epoch 30/180
2625/2625 [==============================] - ETA: 0s - loss: 0.6665 - accuracy: 0.8382
Epoch 00030: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 0.6665 - accuracy: 0.8382
Epoch 31/180
2623/2625 [============================>.] - ETA: 0s - loss: 0.6758 - accuracy: 0.8358
Epoch 00031: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 0.6755 - accuracy: 0.8359
Epoch 32/1

2624/2625 [============================>.] - ETA: 0s - loss: 0.5302 - accuracy: 0.8671
Epoch 00056: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 49s 19ms/step - loss: 0.5301 - accuracy: 0.8671
Epoch 57/180
2625/2625 [==============================] - ETA: 0s - loss: 0.5259 - accuracy: 0.8704
Epoch 00057: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 47s 18ms/step - loss: 0.5259 - accuracy: 0.8704
Epoch 58/180
2625/2625 [==============================] - ETA: 0s - loss: 0.5228 - accuracy: 0.8697
Epoch 00058: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 0.5228 - accuracy: 0.8697
Epoch 59/180
2625/2625 [==============================] - ETA: 0s - loss: 0.5115 - accuracy: 0.8736 ETA
Epoch 00059:

2625/2625 [==============================] - 44s 17ms/step - loss: 0.4522 - accuracy: 0.8883
Epoch 84/180
2625/2625 [==============================] - ETA: 0s - loss: 0.4561 - accuracy: 0.8884 ETA: 0s - los
Epoch 00084: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 44s 17ms/step - loss: 0.4561 - accuracy: 0.8884
Epoch 85/180
2622/2625 [============================>.] - ETA: 0s - loss: 0.4485 - accuracy: 0.8886 ETA: 0s - loss: 0.4463 
Epoch 00085: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 44s 17ms/step - loss: 0.4484 - accuracy: 0.8885
Epoch 86/180
2624/2625 [============================>.] - ETA: 0s - loss: 0.4500 - accuracy: 0.8892
Epoch 00086: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss

Epoch 111/180
2623/2625 [============================>.] - ETA: 0s - loss: 0.4199 - accuracy: 0.8989
Epoch 00111: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 46s 17ms/step - loss: 0.4198 - accuracy: 0.8988
Epoch 112/180
2623/2625 [============================>.] - ETA: 0s - loss: 0.4504 - accuracy: 0.8970
Epoch 00112: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 0.4502 - accuracy: 0.8970
Epoch 113/180
2623/2625 [============================>.] - ETA: 0s - loss: 0.4281 - accuracy: 0.8996
Epoch 00113: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 0.4286 - accuracy: 0.8996
Epoch 114/180
2625/2625 [==============================] - ETA: 0s - loss: 0.4151 - accuracy: 0.8993

2624/2625 [============================>.] - ETA: 0s - loss: 0.4120 - accuracy: 0.9022
Epoch 00138: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 46s 17ms/step - loss: 0.4120 - accuracy: 0.9022
Epoch 139/180
2625/2625 [==============================] - ETA: 0s - loss: 0.4181 - accuracy: 0.9026
Epoch 00139: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 46s 17ms/step - loss: 0.4181 - accuracy: 0.9026
Epoch 140/180
2623/2625 [============================>.] - ETA: 0s - loss: 0.4113 - accuracy: 0.9022
Epoch 00140: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 0.4112 - accuracy: 0.9022
Epoch 141/180
2625/2625 [==============================] - ETA: 0s - loss: 0.4015 - accuracy: 0.9039
Epoch 00141: 

2625/2625 [==============================] - 46s 18ms/step - loss: 0.4057 - accuracy: 0.9054
Epoch 166/180
2624/2625 [============================>.] - ETA: 0s - loss: 0.3781 - accuracy: 0.9096
Epoch 00166: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 0.3781 - accuracy: 0.9096
Epoch 167/180
2623/2625 [============================>.] - ETA: 0s - loss: 0.4296 - accuracy: 0.9026
Epoch 00167: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 0.4301 - accuracy: 0.9025
Epoch 168/180
2623/2625 [============================>.] - ETA: 0s - loss: 0.3830 - accuracy: 0.9090
Epoch 00168: saving model to D:\Desktop\code\BrainBeats\Symphony\src\models\JazzDrums/checkpoints\cp.ckpt
2625/2625 [==============================] - 45s 17ms/step - loss: 0.3829 - accuracy: 0.9091
Epoch 1